In [1]:
import django
import os
import json
os.environ['DJANGO_SETTINGS_MODULE'] = 'django_settings'
django.setup()

In [13]:
import pandas as pd

In [14]:
from data_management.models import *

In [15]:
CroppedImage.objects.select_related('origin_source')

<QuerySet [<CroppedImage: CroppedImage object (178)>, <CroppedImage: CroppedImage object (179)>, <CroppedImage: CroppedImage object (180)>, <CroppedImage: CroppedImage object (181)>, <CroppedImage: CroppedImage object (182)>, <CroppedImage: CroppedImage object (183)>, <CroppedImage: CroppedImage object (184)>, <CroppedImage: CroppedImage object (185)>, <CroppedImage: CroppedImage object (186)>, <CroppedImage: CroppedImage object (187)>, <CroppedImage: CroppedImage object (188)>, <CroppedImage: CroppedImage object (189)>, <CroppedImage: CroppedImage object (190)>, <CroppedImage: CroppedImage object (191)>, <CroppedImage: CroppedImage object (192)>, <CroppedImage: CroppedImage object (193)>, <CroppedImage: CroppedImage object (194)>, <CroppedImage: CroppedImage object (195)>, <CroppedImage: CroppedImage object (196)>, <CroppedImage: CroppedImage object (197)>, '...(remaining elements truncated)...']>

In [43]:
left = []
top = []
right = []
bottom = []
origin_url = []

In [44]:
for c in CroppedImage.objects.all()[:10]:
    left.append(c.left)
    top.append(c.top)
    right.append(c.right)
    bottom.append(c.bottom)
    origin_url.append(c.origin_source.image.url)

In [53]:
dic = {}
dic['left']=left
dic['top']=top
dic['right']=right
dic['bottom']=bottom
dic['origin_url']=origin_url
print(len(dic['left']))

10


In [54]:
test_data_pd=pd.DataFrame.from_dict(dic)

In [55]:
test_data_pd

,left,top,right,bottom,origin_url
0,0.1088,0.0736,0.9796,0.9332,https://monde-data.s3.amazonaws.com/original-b...
1,0.1234,0.0137,0.8547,0.9970,https://monde-data.s3.amazonaws.com/original-b...
2,0.2437,0.1885,0.8038,0.9606,https://monde-data.s3.amazonaws.com/original-b...
3,0.0175,0.2182,0.9817,0.7890,https://monde-data.s3.amazonaws.com/original-b...
4,0.0267,0.0102,0.9762,0.9702,https://monde-data.s3.amazonaws.com/original-b...
5,0.0304,0.2482,0.9654,0.7661,https://monde-data.s3.amazonaws.com/original-b...
6,0.0307,0.4273,0.9474,0.9519,https://monde-data.s3.amazonaws.com/original-b...
7,0.0231,0.1217,0.9788,0.9030,https://monde-data.s3.amazonaws.com/original-b...
8,0.0131,0.0329,1.0000,0.9465,https://monde-data.s3.amazonaws.com/original-b...
9,0.0592,0.4290,0.9167,0.9273,https://monde-data.s3.amazonaws.com/original-b...


In [58]:
test_data_pd.to_csv("test_pd.csv", mode='w', index=False)